初期設定

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!mkdir /root/.kaggle 
!mv kaggle.json /root/.kaggle
!pip install kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


データ初期化

In [0]:
import os
import subprocess
import shutil
import pandas as pd
import glob

CLASS_COUNT_LIMIT = 3
VALIDATION_SPLIT = 0.2
SEED = 1470

BASE_DIR = 'data'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
VALIDATION_DIR = os.path.join(BASE_DIR, 'validation')

def download(clean=False):
    if clean:
       if os.path.exists('data'):
          shutil.rmtree('data')
    if not os.path.exists('data'):
        os.mkdir('data')
    os.chdir('data')
    if not os.path.exists('train.zip'):
        subprocess.call(['kaggle', 'competitions', 'download', '-c', 'humpback-whale-identification'])
    if os.path.exists('test'):
        shutil.rmtree('test')
    os.mkdir('test')
    os.chdir('test')
    subprocess.call(['unzip', '../test.zip', '-d', '0'])
    os.chdir('../')
    if os.path.exists('train'):
        shutil.rmtree('train')
    subprocess.call(['unzip', 'train.zip', '-d', 'train'])
    if os.path.exists('validation'):
        shutil.rmtree('validation')
    os.mkdir('validation')
    os.chdir('../')

def train_validation_split(class_list, validation_split):
    # 5class 未満は全てtrain
    class_count = class_list.count()[0]
    if class_count < CLASS_COUNT_LIMIT:
        return class_list, pd.DataFrame(columns=class_list.columns)
    # 1 or n * validation_split
    validation_count = int(max(class_count * validation_split, 1))
    train_count = class_count - validation_count
    validation_list = class_list.sample(validation_count, random_state=SEED)
    train_list = class_list.drop(validation_list.index)
    return train_list, validation_list

def initialize_train_validation(clean = False):
    # make train and validation data
    # nc = nc > 5 && nc else 0 
    # validation_count = max( 1, nc/10 )
    classes = pd.read_csv('data/train.csv')
    class_value_counts = classes['Id'].value_counts().reset_index()
    # 
    if clean:
        path = os.path.join('./' + TRAIN_DIR, '**/*.txt')
        [shutil.move(x, TRAIN_DIR) for x in glob.glob(path, recursive=True)]
        path = os.path.join('./' + VALIDATION_DIR, '**/*.txt')
        [shutil.move(x, TRAIN_DIR) for x in glob.glob(path, recursive=True)]
        glob.glob(path, recursive=True)
        [shutil.rmtree(os.path.join(TRAIN_DIR, x)) for x in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, x))]
        [shutil.rmtree(os.path.join(VALIDATION_DIR, x)) for x in os.listdir(VALIDATION_DIR) if os.path.isdir(os.path.join(VALIDATION_DIR, x))]
    if not os.path.exists(TRAIN_DIR):
        os.mkdir(TRAIN_DIR)
    if not os.path.exists(VALIDATION_DIR):
        os.mkdir(VALIDATION_DIR)
    for i, v in class_value_counts.iterrows():
        class_name = v[0]
        class_files_list = classes[classes['Id'] == class_name]
        # define class
        # class known whale and new whale
        # # if class_name != 'new_whale':
            # # class_name = 'known_whale
        #if class_name == 'new_whale':
           # unuse_dir = os.path.join(BASE_DIR, 'unuse')
            #os.mkdir(unuse_dir)
            #[shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), unuse_dir) for x in class_files_list.iterrows()]    
            #continue
        train_output_dir = os.path.join(TRAIN_DIR, class_name)
        if not os.path.exists(train_output_dir):
            os.mkdir(train_output_dir)
        validation_output_dir = os.path.join(VALIDATION_DIR, class_name)
        if not os.path.exists(validation_output_dir):
            os.mkdir(validation_output_dir)
        train_list, validation_list = train_validation_split(class_files_list, VALIDATION_SPLIT)
        [shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), validation_output_dir) for x in validation_list.iterrows()]
        [shutil.move(os.path.join(TRAIN_DIR, x[1]['Image']), train_output_dir) for x in train_list.iterrows()]


download(True)
initialize_train_validation(True)

assert(len(os.listdir('./data/validation')) == len(os.listdir('./data/train')))


学習

In [0]:

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import resnet50
from keras import models
from keras import layers
from keras import optimizers
from keras.engine.topology import Input
from keras.callbacks import ModelCheckpoint, CSVLogger

import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

base_dir='data'
train_dir=os.path.join(base_dir, 'train')
validation_dir=os.path.join(base_dir, 'validation')
model_base_dir = 'models/'

# paramater
batch_size=32
SEED=1470
epochs=50

input_size = 224
def get_model():
    main_input = layers.Input(shape=(input_size, input_size, 3,))
    conv_base = resnet50.ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(input_size, input_size, 3))(main_input)
    features = layers.GlobalAveragePooling2D()(conv_base)
    features = layers.BatchNormalization()(features)
    h1 = layers.Dense(4096, activation='relu')(features)
    h1 = layers.BatchNormalization()(h1)
    # new or known
    h2_is_new = layers.Dense(512, activation='relu')(h1)
    h2_is_new = layers.BatchNormalization()(h2_is_new)
    is_new_output = layers.Dense(1, activation='sigmoid', name='is_new_output')(h2_is_new)
    # classify
    h2_classify = layers.Dense(2048, activation='relu')(h1)
    merged = layers.Concatenate()([is_new_output, h2_classify])
    merged = layers.BatchNormalization()(merged)
    output = layers.Dense(5004, activation='softmax', name='output')(merged)
    model = models.Model(inputs=[main_input], outputs=[is_new_output, output])
    model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  loss={'is_new_output': 'binary_crossentropy', 'output' : 'categorical_crossentropy'},
                  loss_weights={'is_new_output': 0.5, 'output': 1.},
                        metrics=['accuracy'])
    return model

def batch_generator(generator):
    for x, y in generator:
        yield x, [y[:,0], y[:,1:]]


In [0]:
# Epoch 00038: val_output_acc did not improve from 0.16531
# Epoch 39/50
# 662/662 [==============================] - 584s 882ms/step - loss: 0.9648 - is_new_output_loss: 0.0666 - output_loss: 0.9315 - is_new_output_acc: 0.9812 - output_acc: 0.6124 - 
# val_loss: 3.1768 - val_is_new_output_loss: 0.7388 - val_output_loss: 2.8074 - val_is_new_output_acc: 0.7510 - val_output_acc: 0.1830

def load_data():
    datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
    )
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(input_size, input_size),
        batch_size=batch_size,
        seed=SEED,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        brightness_range=[0.7, 1.0]
        class_mode='categorical',
    )
    datagen = ImageDataGenerator(
        rescale=1./255,
    )
    val_generator = datagen.flow_from_directory(
        validation_dir,
        target_size=(input_size, input_size),
        batch_size=20,
        seed=SEED,
        class_mode='categorical',
    )
    return train_generator, val_generator





Found 21175 images belonging to 5005 classes.
Found 4186 images belonging to 5005 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/50
662/662 [==============================] - 610s 921ms/step - loss: 5.9041 - is_new_output_loss: 0.7149 - output_loss: 5.5467 - is_new_output_acc: 0.6179 - output_acc: 4.2485e-04 - val_loss: 4.9998 - val_is_new_output_loss: 0.6208 - val_output_loss: 4.6894 - val_is_new_output_acc: 0.6840 - val_output_acc: 0.0000e+00

Epoch 00001: val_output_acc improved from -inf to 0.00000, saving model to ./drive/My Drive/models/separate_dnn/01-5.90-5.00.hdf5
Epoch 2/50
662/662 [==============================] - 581s 878ms/step - loss: 5.6536 - is_new_output_loss: 0.6038 - output_loss: 5.3517 - is_new_output_acc: 0.6945 - output_acc: 0.0027 - val_loss: 4.7966 - val_is_new_output_loss: 0.5601 - val_output_loss: 4.5166 - val_is_new_output_acc: 0.7210 - val_output_acc: 0.0050

Epoch 00002: val_output_acc improved from 0.00000 to 0.00500, saving model to ./drive/My Drive/models/separate_dnn/02-5.65-4.80.hdf5
Epoch 3/50
662/662 [==============================] - 582s 879ms/step - loss: 5.4234 - 

In [0]:
# Epoch 5/100
# 260/260 [==============================] - 661s 3s/step - loss: 6.7437 - is_new_output_loss: 0.7211 - output_loss: 6.0226 - is_new_output_acc: 0.5886 - output_acc: 0.3873 - 
# val_loss: 8.9925 - val_is_new_output_loss: 0.4110 - val_output_loss: 8.5815 - val_is_new_output_acc: 0.9970 - val_output_acc: 0.0030
# Epoch 6/100
def get_model():
    main_input = layers.Input(shape=(input_size, input_size, 3,))
    conv_base = resnet50.ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(input_size, input_size, 3))
    for layer in conv_base.layers:
        layer.trainable = False;
    main_input = conv_base.input
    embedding = conv_base.output
    features = layers.GlobalAveragePooling2D()(embedding)
    # new or known
    hid1 = layers.Dense(512)(features)
    hid1 = layers.Activation('relu')(hid1)
    hid1 = layers.Dropout(0.5)(hid1)
    is_new_output = layers.Dense(1, activation='sigmoid', name='is_new_output')(hid1)
    # classify
    hid2 = layers.Dense(4096)(features)
    hid2 = layers.Activation('relu')(hid2)
    hid2 = layers.Concatenate()([hid1, hid2])
    hid2 = layers.Dropout(0.2)(hid2)
    output = layers.Dense(5005, activation='softmax', name='output')(hid2)
    model = models.Model(inputs=[main_input], outputs=[is_new_output, output])
    model.compile(optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),
                  loss={'is_new_output': 'binary_crossentropy', 'output' : 'categorical_crossentropy'},
                  loss_weights={'is_new_output': 1, 'output': 1.},
                        metrics=['accuracy'])
    return model

In [0]:
train_generator, val_generator = load_data()
model = get_model()

model_dir = os.path.join(model_base_dir, 'separate_dnn2')
model_dir = os.path.join('./drive/My Drive', model_dir)

os.makedirs(model_dir, exist_ok=True)

model_checkpoint_path = os.path.join(model_dir, '{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5')
model_checkpoint = ModelCheckpoint(model_checkpoint_path, monitor='val_output_acc', verbose=1, save_best_only=True)
csv_log_path = os.path.join(model_dir, 'train.log')
csv_logger = CSVLogger(csv_log_path)
history = model.fit_generator(batch_generator(train_generator),
                    steps_per_epoch=len(train_generator),
                    epochs=epochs,
                    validation_data=batch_generator(val_generator),
                    validation_steps=50,
                    callbacks=[model_checkpoint, csv_logger]
                    )